In [ ]:
from ultralytics import YOLO
import mediapipe as mp
import cv2
import torch
import numpy as np
import glob
import pandas as pd
from tensorflow import keras
from keras.models import load_model

In [ ]:

# 세그먼트를 위한 YOLOv8 모델 로드
model = YOLO('yolov8n-seg.pt')
# 모션인식을 위한 모델 로드
gesture = load_model('./data/Mini_Project/motion/dataset/models/model.h5')

# MediaPipe hands model (초기화)
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

hands = mp_hands.Hands(
    max_num_hands = 1,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5)

gesture_start = False  # 제스처 인식 시작 여부를 나타내는 변수
gesture_end = False  # 제스처 인식 종료 여부를 나타내는 변수
gesture_duration = 0  # 제스처 인식 지속 시간을 나타내는 변수 (프레임 수)
is_gesture_detected = False  # 제스처 중첩 인식 방지를 위한 변수

# 배경을 불러오기
folder_path = './data/background'  # 폴더 경로 설정
file_list = glob.glob(folder_path + '/*')  # 폴더 내의 파일 목록 얻기
file_count = len(file_list)  # 파일 개수 계산
idx = 0 # 파일 리스트를 적절히 초기화해야 함
actions = ['Next', 'Preview', 'Cam_Off', 'Cam_On'] # 모션 인식 레이블
seq_length = 30
seq = []
action_seq = []
# 웹캠을 위한 비디오 캡처 객체 열기
cap = cv2.VideoCapture(0)

first_hand_detection = True


# 비디오 프레임을 반복하여 처리
while cap.isOpened():
    # start = time.time()
    # 비디오에서 프레임 읽기
    success, frame = cap.read()
    hand_frame = frame.copy()
    hand_frame = cv2.cvtColor(hand_frame, cv2.COLOR_RGB2BGR)
    # ret, img = cap.read()
    # img0 = img.copy()

    # hand_frame = cv2.flip(hand_frame, 1)
    # hand_frame = cv2.cvtColor(hand_frame, cv2.COLOR_BGR2RGB)
    # result = hands.process(img)
    # img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

    if success:
        # YOLOv8를 사용하여 프레임에 대한 추론 실행
        # classes를 통해 사람만 탐지하도록 설정
        results = model(frame, classes=0, verbose=False)
        handresult = hands.process(hand_frame)
        # handresult = cv
        
        # 검정색 배경 이미지 생성
        background = np.zeros(frame.shape, dtype=np.uint8)
        blackscreen = np.zeros(frame.shape, dtype=np.uint8)
        
        try: # 사람이 탐지되지않으면 오류나는것을 방지
            for result in results:
                # print(result.names) # 탐지 객체 목록 딕셔너리
                # if result.boxes.conf >= 0.80:
                    # print(result.boxes.conf)
                    # if result.probs is not None and result.probs.top1 == 1:
                    for mask in result.masks:
                        # 마스크 데이터에서 차원 축소
                        m = torch.squeeze(mask.data)
                        # 마스크를 RGB 형식으로 변환하여 컴포지트(composite) 텐서 생성
                        composite = torch.stack((m, m, m), 2)
                        # 프레임과 컴포지트를 곱하여 마스크 영역 추출
                        tmp = frame * composite.cpu().numpy().astype(np.uint8)
                        # 추출된 마스크 영역을 배경에 누적
                        # 미리 생성해둔 검은 배경에 세그먼트된 영역을 채워넣음
                        background += tmp
                # else:
                #     # results = model(frame, classes=0)
        except:
            pass
        
        if handresult.multi_hand_landmarks is not None:
            for res in handresult.multi_hand_landmarks:
                joint = np.zeros((21, 4))
                for j, lm in enumerate(res.landmark):
                    joint[j] = [lm.x, lm.y, lm.z, lm.visibility]
                    
            # 점들 간의 각도 계산하기
            v1 = joint[[0,1,2,3,0,5,6,7,0,9,10,11,0,13,14,15,0,17,18,19], :3] # Parent joint
            v2 = joint[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20], :3] # Child joint
            v = v2 - v1 # v2와 v1 사이의 벡터 구하기

            # 점곱을 구한 다음 arccos으로 각도 구하기
            v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]
            
            # Get angle using arcos of dot product
            angle = np.arccos(np.einsum('nt,nt->n',
                v[[0,1,2,4,5,6,8,9,10,12,13,14,16,17,18],:], 
                v[[1,2,3,5,6,7,9,10,11,13,14,15,17,18,19],:])) # [15,]
            angle = np.degrees(angle) # 라디안을 각도로 바꾸기

            d = np.concatenate([joint.flatten(), angle])

            seq.append(d)
            # mp_drawing.draw_landmarks(frame, res, mp_hands.HAND_CONNECTIONS)

            if len(seq) < seq_length:
                continue
            input_data = np.expand_dims(np.array(seq[-seq_length:], dtype=np.float32), axis=0)
            # 모델 예측
            y_pred = gesture.predict(input_data).squeeze()
            # 예측한 값의 인덱스 구하기
            i_pred = int(np.argmax(y_pred))
            conf = y_pred[i_pred]
            # confidence가 0.9보다 작으면
            if conf < 0.7:
                continue # 제스쳐 인식 못 한 상황으로 판단
            action = actions[i_pred]
            action_seq.append(action) # action_seq에 action을 저장
            #print(action_seq)
            # 보인 제스쳐의 횟수가 3 미만인 경우에는 계속
            if len(action_seq) < 3:
                continue
            # 만약 마지막 3개의 제스쳐가 같으면 제스쳐가 제대로 취해졌다고 판단
            if action_seq[-1] == action_seq[-2] == action_seq[-3]:
                this_action = action
                # # print(this_action)
                if action == 'Next':
                    if not is_gesture_detected:  # 중첩 인식 방지 변수 확인
                        idx += 1
                        if idx >= len(file_list):
                            idx = 0
                        is_gesture_detected = True  # 중첩 인식 방지 변수 설정
                elif action == 'Preview':
                    if not is_gesture_detected:  # 중첩 인식 방지 변수 확인
                        idx -= 1
                        if idx < 0:
                            idx = len(file_list) - 1
                        is_gesture_detected = True  # 중첩 인식 방지 변수 설정
                elif action == 'Cam_off':
                    cv2.imshow("Project", blackscreen)
                    is_gesture_detected = True  # 중첩 인식 방지 변수 설정
                elif action == 'cam_On':
                    cv2.imshow("Project", cv2.flip(background, 1)) # 좌우반전
                    is_gesture_detected = True  # 중첩 인식 방지 변수 설정
                else:
                    gesture_start = False  # 제스처 인식 시작 여부를 나타내는 변수
                    gesture_end = False  # 제스처 인식 종료 여부를 나타내는 변수
                    gesture_duration = 0  # 제스처 인식 지속 시간을 나타내는 변수 (프레임 수)
                    is_gesture_detected = False  # 제스처 중첩 인식 방지를 위한 변수
                # elif action == 'Cam_Off':
                #     background_copy = background
                #     background = bgimg_resized
                # elif action == 'Cam_On':
                    # background = background_copy
            
        # 배경 이미지를 원하는 배경으로 마스킹
        bgimg = cv2.imread(file_list[idx])
        bgimg_resized = cv2.resize(bgimg, (frame.shape[1], frame.shape[0]))
        # 검정색배경에 세그먼트된 영상에서 0인값을 설정해둔 배경으로 마스킹
        background = np.where(background == 0, bgimg_resized, background)
        # background = cv2.resize(background, (960,720))
        background = background.astype(np.uint8)
        # 마스킹된 이미지 출력
        if first_hand_detection:
            if handresult.multi_hand_landmarks is None:
                cv2.imshow("Project", blackscreen)
            else:
                first_hand_detection = False
                cv2.imshow("Project", cv2.flip(background, 1)) # 좌우반전
        else:
            cv2.imshow("Project", cv2.flip(background, 1)) # 좌우반전

        key = cv2.waitKey(1)
        # 'q', 'esc', x버튼 키가 눌리면 루프 종료
        if key == 113 or key == 27 or cv2.getWindowProperty('Project', cv2.WND_PROP_VISIBLE) < 1: # ord("q")
            break
        # 'c' 키가 눌리면 다음 배경
        elif key == 99: # ord("c")
            idx += 1
            if idx >= len(file_list):
                idx = 0
        # 'z' 키가 눌리면 이전 배경
        elif key == 122: # ord("z")
            idx -= 1
            if idx < 0:
                idx = len(file_list) - 1
        # end = time.time()
        # print(end - start)
    else:
        # 비디오의 끝에 도달하면 루프 종료
        break

# 비디오 캡처 객체 해제 및 화면 창 닫기
cap.release()
cv2.destroyAllWindows()